In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import asyncio
import os
import json
import nest_asyncio
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv

load_dotenv()

# Patch for Jupyter/Colab
nest_asyncio.apply()

In [2]:
# CELL 1: Mock Data Generator
def generate_cart_abandonment_dataset(n_users: int = 25000) -> pd.DataFrame:
    """Generate realistic cart abandonment dataset"""
    print(f"📊 Generating {n_users:,} cart abandonment records...")

    np.random.seed(42)
    base_date = datetime.now() - timedelta(days=7)
    
    data = {
        'user_id': [f"user_{i:06d}" for i in range(n_users)],
        'cart_abandoned_date': [
            (base_date + timedelta(days=np.random.randint(0, 8))).strftime('%Y-%m-%d')
            for _ in range(n_users)
        ],
        'last_order_date': [
            None if np.random.random() < 0.3 else
            (base_date - timedelta(days=np.random.randint(1, 365))).strftime('%Y-%m-%d')
            for _ in range(n_users)
        ],
        'avg_order_value': np.round(np.random.lognormal(mean=6.5, sigma=0.8, size=n_users) * 10, 2),
        'sessions_last_30d': np.random.poisson(lam=8, size=n_users),
        'num_cart_items': np.random.choice(range(1, 11), size=n_users, p=[0.3,0.2,0.15,0.1,0.08,0.07,0.05,0.03,0.01,0.01]),
        'engagement_score': np.random.beta(2, 5, size=n_users),
        'profitability_score': np.random.beta(3, 3, size=n_users)
    }
    
    df = pd.DataFrame(data)
    print(f"✅ Dataset generated: {len(df):,} records")
    return df

In [3]:
# CELL 2: Define Universe
def define_universe(df: pd.DataFrame) -> pd.DataFrame:
    """Select users who abandoned carts in last 7 days"""
    df['cart_abandoned_date'] = pd.to_datetime(df['cart_abandoned_date'])
    now = pd.Timestamp.now()
    universe = df[df['cart_abandoned_date'] >= (now - timedelta(days=7))].copy()
    print(f"🎯 Universe defined: {len(universe):,} users abandoned cart in last 7 days")
    return universe

In [4]:
# CELL 3: Rule-Based Bucketing
def create_bucket_combinations(universe_df: pd.DataFrame) -> pd.DataFrame:
    """Create MECE bucket combinations"""
    print("🔧 Creating MECE bucket combinations...")

    aov_high = round(universe_df['avg_order_value'].quantile(0.8), -2)
    aov_mid = round(universe_df['avg_order_value'].quantile(0.4), -2)
    engagement_high = round(universe_df['engagement_score'].quantile(0.6), 2)
    profitability_high = round(universe_df['profitability_score'].quantile(0.6), 2)

    print(f"📊 Thresholds: AOV_high=${aov_high:,.0f}, AOV_mid=${aov_mid:,.0f}, Eng>={engagement_high}, Profit>={profitability_high}")

    def get_aov_bucket(aov):
        if pd.isna(aov): return "Other"
        elif aov >= aov_high: return "High"
        elif aov >= aov_mid: return "Medium"
        else: return "Low"

    def get_engagement_bucket(eng):
        if pd.isna(eng): return "Other"
        return "High" if eng >= engagement_high else "Low"

    def get_profitability_bucket(prof):
        if pd.isna(prof): return "Other"
        return "High" if prof >= profitability_high else "Low"

    universe_df['aov_bucket'] = universe_df['avg_order_value'].apply(get_aov_bucket)
    universe_df['engagement_bucket'] = universe_df['engagement_score'].apply(get_engagement_bucket)
    universe_df['profitability_bucket'] = universe_df['profitability_score'].apply(get_profitability_bucket)
    universe_df['bucket_combo'] = universe_df['aov_bucket'] + "_" + universe_df['engagement_bucket'] + "_" + universe_df['profitability_bucket']

    print(f"✅ MECE buckets created — {universe_df['bucket_combo'].nunique()} unique combinations")
    return universe_df

In [5]:
# CELL 4: Pydantic Models for Structured LLM Outputs
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

class SegmentResponse(BaseModel):
    """Structured output for jury and judge"""
    segment_name: str = Field(..., description="Marketer-friendly segment name, e.g., 'Premium_Engaged_Profitable'")
    rule: str = Field(..., description="MECE rule in format: 'AOV_High & Engagement_High & Profitability_High'")

class JurySubmission(BaseModel):
    """Single jury member's submission"""
    role: str
    segment: SegmentResponse

class JudgeDecision(BaseModel):
    """Judge's final decision"""
    chosen_segment: SegmentResponse
    reasoning: Optional[str] = Field(None, description="Brief reason for choice (optional)")

In [6]:
# CELL 5: LangChain + Groq Setup
import os
from groq import Groq
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not set — required for LangChain + Groq")

# Initialize LangChain LLMs (sync for simplicity)
llm_creative = ChatGroq(model="qwen/qwen3-32b", temperature=0.3, groq_api_key=GROQ_API_KEY)
llm_strategist = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct", temperature=0.3, groq_api_key=GROQ_API_KEY)
llm_data_sci = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3, groq_api_key=GROQ_API_KEY)
llm_risk = ChatGroq(model="moonshotai/kimi-k2-instruct-0905", temperature=0.2, groq_api_key=GROQ_API_KEY)
llm_judge = ChatGroq(model="openai/gpt-oss-120b", temperature=0.1, groq_api_key=GROQ_API_KEY)  # Use strongest for judge

In [7]:
# CELL 6: Jury Prompt & Output Parser
jury_parser = PydanticOutputParser(pydantic_object=SegmentResponse)

JURY_PROMPT_TEMPLATE = ChatPromptTemplate.from_template("""
You are a {persona}. Generate a SINGLE, MARKETER-FRIENDLY segment name AND rule for users with this profile:

Bucket Combo: {bucket_combo}

Context:
- High/Medium/Low AOV = Average Order Value tiers
- High/Low Engagement = User engagement with brand  
- High/Low Profitability = Customer profitability score

Rules:
- Must be MECE-compliant (no overlaps, fully exhaustive)
- Segment Name: Use clear, business-friendly naming (e.g., "Premium_Engaged_Profitable")
- Rule: Describe logic in format: "AOV_[Tier] & Engagement_[Tier] & Profitability_[Tier]"
- Do NOT include thresholds or technical terms in segment name
- Keep segment name under 4 words, use underscores
- {format_instructions}

Output ONLY valid JSON. No explanation.
""")

# Define jury configs
JURY_CONFIGS = {
    "creative_marketer": {
        "llm": llm_creative,
        "persona": "creative marketer focused on catchy, memorable segment names"
    },
    "growth_strategist": {
        "llm": llm_strategist,
        "persona": "growth strategist focused on scalable, actionable segments"
    },
    "data_scientist": {
        "llm": llm_data_sci,
        "persona": "data scientist focused on precise, technical segment definitions"
    },
    "risk_manager": {
        "llm": llm_risk,
        "persona": "risk-averse manager focused on safe, conservative segment names"
    }
}

In [8]:
# CELL 7: Judge Prompt & Output Parser
judge_parser = PydanticOutputParser(pydantic_object=JudgeDecision)

JUDGE_PROMPT_TEMPLATE = ChatPromptTemplate.from_template("""
You are the Chief Segmentation Officer. Pick the BEST segment name and rule for: {bucket_combo}

CRITERIA:
1. MECE Compliance (30%) — Rule must be mutually exclusive, collectively exhaustive
2. Marketing Clarity (25%) — Segment name must be instantly understandable
3. Business Structure (20%) — Segment name should follow "Tier_Attribute_Quality" pattern
4. Rule Precision (15%) — Rule must exactly match bucket combo logic
5. Memorability (10%) — Segment name should be short, catchy, easy to remember

JURY SUBMISSIONS (JSON format):
{jury_submissions_json}

{format_instructions}

""")

In [9]:
# CELL 8: LLM Jury + Judge Functions
import json
from typing import Dict, List

def call_jury_member(role: str, bucket_combo: str) -> JurySubmission:
    """Call one jury member via LangChain"""
    config = JURY_CONFIGS[role]
    
    # Create chain
    chain = JURY_PROMPT_TEMPLATE | config["llm"] | jury_parser
    
    try:
        result = chain.invoke({
            "persona": config["persona"],
            "bucket_combo": bucket_combo,
            "format_instructions": jury_parser.get_format_instructions()
        })
        print(f"✅ {role}: {result.segment_name} | Rule: {result.rule}")
        return JurySubmission(role=role, segment=result)
    except Exception as e:
        print(f"⚠️  Error in {role}: {e} — using fallback")
        fallback_name = f"Fallback_{role}_{bucket_combo}"
        fallback_rule = f"AOV_{bucket_combo.split('_')[0]} & Engagement_{bucket_combo.split('_')[1]} & Profitability_{bucket_combo.split('_')[2]}"
        return JurySubmission(
            role=role,
            segment=SegmentResponse(segment_name=fallback_name, rule=fallback_rule)
        )

def call_judge(bucket_combo: str, jury_submissions: List[JurySubmission]) -> SegmentResponse:
    """Call judge via LangChain"""
    # Convert jury submissions to JSON
    jury_json = json.dumps([{
        "role": js.role,
        "segment_name": js.segment.segment_name,
        "rule": js.segment.rule
    } for js in jury_submissions], indent=2)
    
    # Create chain
    chain = JUDGE_PROMPT_TEMPLATE | llm_judge | judge_parser
    
    try:
        result = chain.invoke({
            "bucket_combo": bucket_combo,
            "jury_submissions_json": jury_json,
            "format_instructions": judge_parser.get_format_instructions()
        })
        print(f"⚖️  Judge Selected: {result.chosen_segment.segment_name} | Rule: {result.chosen_segment.rule}")
        return result.chosen_segment
    except Exception as e:
        print(f"⚠️  Judge Error: {e} — using highest scored fallback")
        # Fallback: pick first submission
        return jury_submissions[0].segment if jury_submissions else SegmentResponse(
            segment_name=f"Segment_{bucket_combo}",
            rule=f"AOV_{bucket_combo.split('_')[0]} & Engagement_{bucket_combo.split('_')[1]} & Profitability_{bucket_combo.split('_')[2]}"
        )

In [10]:
# CELL 6: Apply Size Constraints Function
# Enforces min 500, max 20,000 segment sizes with "Catch_All_Other" folding

def apply_size_constraints(
    universe_df: pd.DataFrame, 
    segment_name_map: Dict[str, str], 
    min_size: int = 100, 
    max_size: int = 500
) -> pd.DataFrame:
    """
    Apply campaign feasibility size constraints to segments.
    
    Args:
        universe_df: DataFrame with bucket_combo column
        segment_name_map: Mapping from bucket_combo to segment names
        min_size: Minimum users per segment (default: 500)
        max_size: Maximum users per segment (default: 20,000)
    
    Returns:
        DataFrame with 'final_segment' column and size constraints applied
    
    Business Logic:
        - Segments <500 users: Too small for campaign efficiency
        - Segments >20,000 users: Too large for personalized targeting
        - Invalid segments folded into "Catch_All_Other" for operational handling
    """
    
    print("🔧 APPLYING SIZE CONSTRAINTS")
    print("=" * 45)
    
    # Create copy to avoid modifying original
    df = universe_df.copy()
    
    # Map bucket combinations to segment names
    df['final_segment'] = df['bucket_combo'].map(segment_name_map)
    
    # Handle any unmapped buckets (edge case protection)
    unmapped_count = df['final_segment'].isna().sum()
    if unmapped_count > 0:
        print(f"⚠️  {unmapped_count} users with unmapped bucket combinations")
        df.loc[df['final_segment'].isna(), 'final_segment'] = 'Unmapped_Segment'
    
    # Calculate segment sizes
    segment_sizes = df['final_segment'].value_counts()
    print(f"📊 Initial segment distribution:")
    for segment, size in segment_sizes.head(10).items():
        print(f"   {segment:30} | {size:5,} users")
    
    if len(segment_sizes) > 10:
        print(f"   ... and {len(segment_sizes) - 10} more segments")
    
    # Identify segments violating size constraints
    too_small = segment_sizes[segment_sizes < min_size].index.tolist()
    too_large = segment_sizes[segment_sizes > max_size].index.tolist()
    invalid_segments = too_small + too_large
    
    print(f"\n🎯 SIZE CONSTRAINT ANALYSIS:")
    print(f"   Min size required: {min_size:,} users")
    print(f"   Max size allowed: {max_size:,} users")
    print(f"   Segments too small (<{min_size:,}): {len(too_small)}")
    print(f"   Segments too large (>{max_size:,}): {len(too_large)}")
    print(f"   Total invalid segments: {len(invalid_segments)}")
    
    if too_small:
        print(f"\n⚠️  TOO SMALL SEGMENTS (will be folded):")
        for segment in too_small:
            size = segment_sizes[segment]
            print(f"   {segment:30} | {size:5,} users")
    
    if too_large:
        print(f"\n⚠️  TOO LARGE SEGMENTS (will be folded):")
        for segment in too_large:
            size = segment_sizes[segment]
            print(f"   {segment:30} | {size:5,} users")
    
    # Apply size constraints by folding invalid segments
    if invalid_segments:
        users_to_fold = df[df['final_segment'].isin(invalid_segments)].shape[0]
        print(f"\n🔄 FOLDING {users_to_fold:,} users into 'Catch_All_Other'")
        
        # Fold invalid segments into catch-all
        df.loc[df['final_segment'].isin(invalid_segments), 'final_segment'] = 'Catch_All_Other'
    else:
        print("\n✅ All segments meet size constraints - no folding needed")
    
    # Final segment distribution
    final_segment_sizes = df['final_segment'].value_counts()
    print(f"\n📈 FINAL SEGMENT DISTRIBUTION:")
    for segment, size in final_segment_sizes.items():
        valid_status = "✅ Valid" if min_size <= size <= max_size else "⚠️  Check"
        print(f"   {segment:30} | {size:5,} users | {valid_status}")
    
    # Validation check
    total_users = len(df)
    total_segmented = final_segment_sizes.sum()
    
    print(f"\n🔍 VALIDATION:")
    print(f"   Original universe: {total_users:,} users")
    print(f"   Final segmented: {total_segmented:,} users")
    print(f"   Coverage: {(total_segmented/total_users)*100:.1f}%")
    print(f"   Unique segments: {df['final_segment'].nunique()}")
    
    # Check for any remaining constraint violations
    violations = final_segment_sizes[(final_segment_sizes < min_size) | (final_segment_sizes > max_size)]
    if len(violations) > 0:
        print(f"⚠️  WARNING: {len(violations)} segments still violate constraints!")
        for segment, size in violations.items():
            print(f"   {segment}: {size:,} users")
    else:
        print("✅ All final segments meet size constraints")
    
    print(f"\n✅ Size constraints applied successfully")
    return df

In [11]:
# CELL 9: Generate Segment Names + Rules
def generate_segment_names_with_langchain_jury(universe_df: pd.DataFrame) -> tuple:
    """Generate segment names and rules using LangChain + Groq"""
    print("🧠 INITIALIZING LANGCHAIN LLM JURY + JUDGE SYSTEM")
    
    unique_buckets = universe_df['bucket_combo'].unique()
    segment_name_map = {}
    segment_rule_map = {}

    for i, bucket in enumerate(unique_buckets, 1):
        print(f"\n--- Processing Bucket {i}/{len(unique_buckets)}: {bucket} ---")

        try:
            # Call jury members
            jury_submissions = [
                call_jury_member(role, bucket)
                for role in JURY_CONFIGS.keys()
            ]
            
            # Call judge
            judge_decision = call_judge(bucket, jury_submissions)
            
            segment_name_map[bucket] = judge_decision.segment_name
            segment_rule_map[bucket] = judge_decision.rule
            
        except Exception as e:
            print(f"⚠️  Error processing bucket {bucket}: {e}")
            segment_name_map[bucket] = f"Segment_{bucket.replace('_', '')}"
            segment_rule_map[bucket] = f"AOV_{bucket.split('_')[0]} & Engagement_{bucket.split('_')[1]} & Profitability_{bucket.split('_')[2]}"

    print(f"✅ Generated {len(segment_name_map)} segment names and rules")
    return segment_name_map, segment_rule_map

In [12]:
# CELL 10 : Scores Compuation
def compute_segment_scores(universe_df: pd.DataFrame, bucket_combo_to_rule_map: Dict[str, str]) -> pd.DataFrame:
    """
    Compute multi-dimensional scores for each segment.
    Uses bucket_combo to look up rules — NOT final_segment.
    """
    print("📈 Computing segment scores...")

    results = []
    total_universe = len(universe_df)
    min_size = 20   # ✅ Fixed: Define min_size
    max_size = 200 # ✅ Fixed: Define max_size

    # Get unique final segments
    for segment in universe_df['final_segment'].unique():
        # Get all users in this segment
        seg_df = universe_df[universe_df['final_segment'] == segment].copy()  
        size = len(seg_df)

        if size == 0:
            continue

        seg_df.loc[:, 'days_since_abandon'] = (pd.Timestamp.now() - seg_df['cart_abandoned_date']).dt.days + 1

        # 1. conversion_potential = engagement_score * (1 / days_since_abandonment)
        conv_potential = (seg_df['engagement_score'] * (1 / seg_df['days_since_abandon'])).mean()

        # 2. lift_vs_control = mock random between 0.5 and 1.0 (seeded for reproducibility)
        np.random.seed(hash(segment) % 2**32)
        lift_vs_control = np.random.uniform(0.5, 1.0)

        # 3. size (normalized)
        norm_size = size / total_universe

        # 4. profitability = mean of profitability_score
        profitability = seg_df['profitability_score'].mean()

        # 5. strategic_fit = (conversion_potential + profitability) / 2
        strategic_fit = (conv_potential + profitability) / 2

        # 6. overall_score = weighted sum
        weights = [0.3, 0.2, 0.1, 0.3, 0.1]  # Conv, Lift, Size, Profit, Strat
        overall_score = (
            weights[0] * conv_potential +
            weights[1] * lift_vs_control +
            weights[2] * norm_size +
            weights[3] * profitability +
            weights[4] * strategic_fit
        )

        # Sample one user to get their bucket_combo
        sample_user = seg_df.iloc[0] if len(seg_df) > 0 else None
        if sample_user is not None and 'bucket_combo' in seg_df.columns:
            bucket_combo = sample_user['bucket_combo']
            rules = bucket_combo_to_rule_map.get(bucket_combo, "Catch_All_Other")
        else:
            rules = "Catch_All_Other"

        # Append to results
        results.append({
            'Segment Name': segment,
            'Rules Applied': rules, 
            'Size': size,
            'Conv_Pot': round(conv_potential, 2),
            'Lift_vs_Control': round(lift_vs_control, 2),
            'Profitability': round(profitability, 2),
            'Strategic_Fit': round(strategic_fit, 2),
            'Overall_Score': round(overall_score, 2),
            'Valid': 'Yes' if min_size <= size <= max_size else 'No'
        })

    # Create results DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Overall_Score', ascending=False).reset_index(drop=True)

    print(f"✅ Computed scores for {len(results_df)} segments")
    return results_df

In [13]:
# CELL 11 : Main Workflow (Updated)
def main():
    print("="*70)
    print("🚀 STARTING MECE CART ABANDONER SEGMENTATION — LANGCHAIN + GROQ")
    print("="*70)

    # 1. Generate data
    df = generate_cart_abandonment_dataset(2500)
    universe = define_universe(df)

    # 2. Create buckets
    universe = create_bucket_combinations(universe)

    # 3. LLM Jury + Judge — LANGCHAIN
    try:
        segment_name_map, segment_rule_map = generate_segment_names_with_langchain_jury(universe)
    except Exception as e:
        print(f"❌ LLM system failed: {e}")
        return None, None, None

    # 4. Apply constraints
    universe = apply_size_constraints(universe, segment_name_map)

    # 5. Compute scores (pass segment_rule_map)
    results_df = compute_segment_scores(universe, segment_rule_map)

 
    # 6. Display sample output
    print("\n📋 SAMPLE OUTPUT (TOP 5 SEGMENTS):")
    print(results_df.head().to_string(index=False))

    print(f"\n🎉 COMPLETED — {len(results_df)} segments ready for marketing campaigns!")
    return universe, results_df, segment_rule_map

In [16]:
# CELL 12: Run Sample End
print("🚀 Starting workflow...")
try:
    universe_df, results_df, segment_rule_map = main()
    if results_df is not None:
        print("\n✨ SUCCESS! Your MECE segmentation system is complete.")
        print("\n📄 Final CSV Columns:")
        print(results_df.columns.tolist())
except Exception as e:
    print(f"\n❌ WORKFLOW FAILED: {e}")

🚀 Starting workflow...
🚀 STARTING MECE CART ABANDONER SEGMENTATION — LANGCHAIN + GROQ
📊 Generating 2,500 cart abandonment records...
✅ Dataset generated: 2,500 records
🎯 Universe defined: 2,171 users abandoned cart in last 7 days
🔧 Creating MECE bucket combinations...
📊 Thresholds: AOV_high=$13,000, AOV_mid=$5,500, Eng>=0.32, Profit>=0.56
✅ MECE buckets created — 12 unique combinations
🧠 INITIALIZING LANGCHAIN LLM JURY + JUDGE SYSTEM

--- Processing Bucket 1/12: Medium_Low_Low ---
⚠️  Error in creative_marketer: Invalid json output: <think>
Okay, let's tackle this. The user wants a segment name and rule for the Medium_Low_Low bucket. First, I need to map each tier to appropriate adjectives. The AOV is Medium, so maybe "Mid" or "Moderate". Engagement is Low, so "Disengaged" makes sense. Profitability is also Low, so "Non-Profitable" could work.

Now, the segment name should be concise and under four words. Let me combine those adjectives. "Mid_Disengaged_NonProfitable" seems to fit. Che

✅ growth_strategist: Low_Value_Low_Engaged_High_Profit | Rule: AOV_Low & Engagement_Low & Profitability_High
✅ data_scientist: Low_Value_High | Rule: AOV_Low & Engagement_Low & Profitability_High
✅ risk_manager: Budget_Watchers | Rule: AOV_Low & Engagement_Low & Profitability_High
⚖️  Judge Selected: Low_Value_Low_Engaged_High_Profit | Rule: AOV_Low & Engagement_Low & Profitability_High

--- Processing Bucket 3/12: Medium_High_Low ---
⚠️  Error in creative_marketer: Invalid json output: <think>
Okay, let's tackle this. The user wants a segment name and rule for the Medium_High_Low bucket combo. The context gives us three tiers: AOV (Medium), Engagement (High), Profitability (Low). 

First, the segment name needs to be marketer-friendly, under four words with underscores. Let's break down each component. 

AOV is Medium. So maybe "Mid" as a prefix? But "Medium" might be clearer. Engagement is High, so "Engaged" is a common term. Profitability is Low, so "Low_Profit" or "Low_Profitabilit

✅ growth_strategist: High_Value_Engaged | Rule: AOV_Medium_High & Engagement_High & Profitability_High
✅ data_scientist: Medium_Engaged_Profitable | Rule: AOV_Medium & Engagement_High & Profitability_High
✅ risk_manager: Steady_Core_Value | Rule: AOV_Medium & Engagement_High & Profitability_High
⚖️  Judge Selected: Medium_Engaged_Profitable | Rule: AOV_Medium & Engagement_High & Profitability_High

--- Processing Bucket 5/12: High_Low_Low ---
⚠️  Error in creative_marketer: Invalid json output: <think>
Okay, let's tackle this. The user wants a segment name and rule for the High_Low_Low bucket combo. The context is AOV tiers, engagement, and profitability. The rules need to be MECE, so no overlaps.

First, the segment name should be business-friendly and under four words with underscores. The example given is "Premium_Engaged_Profitable". Since the combo is High for AOV, Low for Engagement, and Low for Profitability, I need to find terms that represent each tier.

High AOV could be "Hig

✅ growth_strategist: Mid_Value_Low_Active | Rule: AOV_Medium & Engagement_Low & Profitability_High
✅ data_scientist: Medium_Engaged_Profitable | Rule: AOV_Medium & Engagement_Medium & Profitability_High
✅ risk_manager: Steady_Core | Rule: AOV_Medium & Engagement_Low & Profitability_High
⚖️  Judge Selected: Medium_Low_High | Rule: AOV_Medium & Engagement_Low & Profitability_High

--- Processing Bucket 9/12: High_Low_High ---
⚠️  Error in creative_marketer: Invalid json output: <think>
Okay, let's tackle this. The user wants a segment name and rule for the High_Low_High bucket combo. The context gives us three tiers each for AOV, Engagement, and Profitability. The rules need to be MECE, so no overlaps.

First, the segment name should be marketer-friendly and under four words with underscores. The example given is "Premium_Engaged_Profitable". So I need to think of adjectives that represent High, Low, High in that order for AOV, Engagement, Profitability.

High AOV could be "Premium" or "

✅ growth_strategist: Low_Value_Customers | Rule: AOV_Low & Engagement_Low & Profitability_Low
✅ data_scientist: Low_Value_Customers | Rule: AOV_Low & Engagement_Low & Profitability_Low
✅ risk_manager: Budget_Passive_Costly | Rule: AOV_Low & Engagement_Low & Profitability_Low
⚖️  Judge Selected: Low_Passive_Costly | Rule: AOV_Low & Engagement_Low & Profitability_Low

--- Processing Bucket 12/12: High_High_Low ---
⚠️  Error in creative_marketer: Invalid json output: <think>
Okay, let's tackle this. The user wants a segment name and rule for the High_High_Low bucket combo. The context gives us three dimensions: AOV (High/Medium/Low), Engagement (High/Low), and Profitability (High/Low). The specific combination here is High AOV, High Engagement, and Low Profitability.

First, the segment name needs to be marketer-friendly, under four words with underscores. The example given is "Premium_Engaged_Profitable". So for High AOV, maybe "Premium" or "High_Value". High Engagement could be "Engaged